<a href="https://colab.research.google.com/github/Senbonsakurakageyoshi1/Senbonsakurakageyoshi1/blob/main/denoiser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 41 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (5,652 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [6]:
!pip install pytesseract

In [24]:
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python

     |████████████████████████████████| 60.5 MB 1.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [10]:
!pip install PYMUPDF

     |████████████████████████████████| 8.8 MB 5.9 MB/s 


In [41]:
!pip install --ignore-installed Pillow==9.0.0

     |████████████████████████████████| 4.3 MB 6.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [45]:
import pytesseract
from pytesseract import Output
import cv2
import fitz
import datetime
import time
import os
import json
import imutils
import numpy as np

In [62]:
import pytesseract
import shutil
import os
import random
try:
    from PIL import Image
except ImportError:
    import Image

In [63]:
path_image = '/content/images'

In [132]:

def extract_text(path):
    """ Fonction d'extraction du texte sur des pdf scannés """
    pages = fitz.open(path)
    
    all_pages =  {}
    all_pages['Image'] = []
    
    les_pages = {}
    
    for page in pages:
        deb = datetime.datetime.now()
        les_mots = []
        # extraire l'image de la page en cours
        # puis du texte de cette image..
        
        
        att = path_image + "/page-" + \
            str(page.number) + "-" + str(time.time()) + ".png"

        mat = fitz.Matrix(2, 2)
        pix = page.getPixmap(matrix=mat)  # render page to an image
        pix.writePNG(att)  # store image as a PNG
        
        # ici pytesseract ..
        img = cv2.imread(att)
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        cv2.threshold(gray_img, 0,255,cv2.THRESH_BINARY| cv2.THRESH_OTSU)[1]

        d = pytesseract.image_to_data(gray_img, output_type=Output.DICT, config='--psm 10')
        #d = pytesseract.image_to_string(gray_img, config=custom_config)
        print(d)
        n_boxes = len(d['text'])
        for i in range(n_boxes):
            (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
            coordonnees = f"{x}-{y}-{w}-{h}"
            mot = d['text'][i]
            if len(mot.strip()) > 0:
                les_mots.append({coordonnees: mot})
                #les_mots.append(mot)

        les_pages[f"Page {page.number}"] = les_mots
        fin = datetime.datetime.now()

        print(f"Diff Excecu Page-{page.number} -> {fin-deb}")
    
    all_pages["Image"].append(les_pages)
    
    return all_pages

In [133]:
path='/content/images/page-0-1651246538.5197868.png'

In [134]:

    
deb = datetime.datetime.now()
all_pages = extract_text(path)
fin = datetime.datetime.now()
print("Diff Excecu = ", fin-deb)
print("Coordonnees des Mots de pages = \n", all_pages)

Deprecation: 'getPixmap' removed from class 'Page' after v1.19 - use 'get_pixmap'.
Deprecation: 'writePNG' removed from class 'Pixmap' after v1.19 - use 'save'.


{'level': [1, 2, 3, 4, 5], 'page_num': [1, 1, 1, 1, 1], 'block_num': [0, 1, 1, 1, 1], 'par_num': [0, 0, 1, 1, 1], 'line_num': [0, 0, 0, 1, 1], 'word_num': [0, 0, 0, 0, 1], 'left': [0, 52, 52, 52, 52], 'top': [0, 7, 7, 7, 7], 'width': [1785, 1688, 1688, 1688, 1688], 'height': [2523, 2486, 2486, 2486, 2486], 'conf': ['-1', '-1', '-1', '-1', 55], 'text': ['', '', '', '', 'i-']}
Diff Excecu Page-0 -> 0:00:01.507021
Diff Excecu =  0:00:01.511629
Coordonnees des Mots de pages = 
 {'Image': [{'Page 0': [{'52-7-1688-2486': 'i-'}]}]}


In [84]:
def denoiser(path):
  image = cv2.imread(path)
  print(image.shape)	
  gray  = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY )
  image = cv2.threshold(gray, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
  dist = cv2.distanceTransform(image, cv2.DIST_L2, 5)
  print(dist)
  dist = cv2.normalize(dist, dist, 0, 1.0, cv2.NORM_MINMAX)
  dist = (dist * 255).astype("uint8")
  dist = cv2.threshold(dist, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
  kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
  opening = cv2.morphologyEx(dist, cv2.MORPH_OPEN, kernel)
  cnts = cv2.findContours(opening.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(cnts)
  chars = []
  for c in cnts:
	# compute the bounding box of the contour
	   (x, y, w, h) = cv2.boundingRect(c)
	# check if contour is at least 35px wide and 100px tall, and if
	# so, consider the contour a digit
	   if w >= 35 and h >= 100:
		     chars.append(c)
  # compute the convex hull of the characters
  chars = np.vstack([chars[i] for i in range(0, len(chars))])
  hull = cv2.convexHull(chars)
  # allocate memory for the convex hull mask, draw the convex hull on
  # the image, and then enlarge it via a dilation
  mask = np.zeros(image.shape[:2], dtype="uint8")
  cv2.drawContours(mask, [hull], -1, 255, -1)
  mask = cv2.dilate(mask, None, iterations=2)
  
  # take the bitwise of the opening image and the mask to reveal *just*
  # the characters in the image
  final = cv2.bitwise_and(opening, opening, mask=mask)
 


  #image = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
  #cv2.resize(image, None, fx=1.2, fy=1.2, interpolation=cv2.INTER_CUBIC)
  cv2.imwrite('reult1.jpg',image)

In [113]:
def denoiser(path):
  image = cv2.imread(path)
  print(image.shape)	
  gray  = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY )
  image = cv2.threshold(gray, 0, 255,	cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
  
  cv2.imwrite('reult1.jpg',image)

In [114]:
denoiser(path)

(1682, 1190, 3)


In [137]:
extractedInformation = pytesseract.image_to_string('/content/reult1.jpg')

In [100]:
extractedInformation1 = pytesseract.image_to_string('/content/reult.jpg')

In [136]:
print(extractedInformation)

fe



In [157]:
extractedInformation1 = pytesseract.image_to_string('/content/images/page-0-1651246538.5197868.png')

In [165]:
formation2 =  pytesseract.image_to_string('/content/oi.png',config='--psm 10')

In [169]:
!tesseract --help-extra

Usage:
  tesseract --help | --help-extra | --help-psm | --help-oem | --version
  tesseract --list-langs [--tessdata-dir PATH]
  tesseract --print-parameters [options...] [configfile...]
  tesseract imagename|imagelist|stdin outputbase|stdout [options...] [configfile...]

OCR options:
  --tessdata-dir PATH   Specify the location of tessdata path.
  --user-words PATH     Specify the location of user words file.
  --user-patterns PATH  Specify the location of user patterns file.
  -l LANG[+LANG]        Specify language(s) used for OCR.
  -c VAR=VALUE          Set value for config variables.
                        Multiple -c arguments are allowed.
  --psm NUM             Specify page segmentation mode.
  --oem NUM             Specify OCR Engine mode.
NOTE: These options must occur before any configfile.

Page segmentation modes:
  0    Orientation and script detection (OSD) only.
  1    Automatic page segmentation with OSD.
  2    Automatic page segmentation, but no OSD, or OCR.
  3    F

In [167]:
print(formation2)

icceay



In [158]:
print(extractedInformation1)

  
 
  
    
    
   
  
 
   
     
  

Monsieur DECOUVELAERE Jean-Francois

14 rue du Marais

44000 NANTES
Nos Réf + (28074485
Affaire :(DECOUVELAERE JEAN FRANCOIS
Véhicule : RENAULT

Immatriculé :\51UuBBF44

Nantes, le 25 mai 2009

RAPPORT D'EXPERTISE

   

Monsieur,

Nous vous remercions pour la confiance que vous nous avez
témoignée en nous transmettant le dossier cité en référence et vous

prions de trouver, ci-dessous, le déroulement de cette affaire et nos
conclusions.

1 NATURE DU LITIGE

La RENAULT Laguna de Monsieur| DECOUVELAERE| est immobilisée au
Garage| LEROUX/a ANCENIS suite A une Panne moteur.

    

  

COLA EXPENTISE - Cabitte( BALDERT & EESCANNE - S.A.RE au canitalde SO 808 SMEET a Tee 7GL aT

